In [73]:
import pandas as pd
import ast
import re
import json
#from difflib import SequenceMatcher
#from geopy.distance import geodesic
#import numpy as np
import numpy as np
from urllib.parse import urlparse
import time

In [74]:
# Import sample data
sample_df = pd.read_parquet('project_b_samples_2k.parquet')
sample_df

,id,base_id,sources,names,categories,confidence,websites,socials,emails,phones,...,base_sources,base_names,base_categories,base_confidence,base_websites,base_socials,base_emails,base_phones,base_brand,base_addresses
0,08f44f055a9a016e0390f050aa3c93c0,1688849865669487,"[{""property"":"""",""dataset"":""meta"",""record_id"":""...","{""primary"":""Goin' Postal Jacksonville""}","{""primary"":""shipping_center"",""alternate"":[""fre...",0.996261,"[""http://www.goinpostaljacksonville.com/""]","[""https://www.facebook.com/463273470392736""]",None,"[""+19049989600""]",...,"[{""dataset"":""Microsoft"",""record_id"":""168884986...","{""primary"":""Goin' Postal Jacksonville""}","{""primary"":""vehicle_shipping"",""alternate"":[""co...",0.77,"[""https://www.goinpostaljacksonville.com/""]",None,None,"[""9049989600""]","{""names"":{}}","[{""freeform"":""7643 Gate Pkwy Ste 104"",""localit..."
1,08f29a456e42e5830324637954145c50,1125899907111860,"[{""property"":"""",""dataset"":""meta"",""record_id"":""...","{""primary"":""Valley Transmission""}","{""primary"":""automotive_repair"",""alternate"":[""a...",0.996261,"[""http://valleytransca.com/""]","[""https://www.facebook.com/107150256087765""]",None,"[""+16194474353""]",...,"[{""dataset"":""Microsoft"",""record_id"":""112589990...","{""primary"":""Valley Transmission""}","{""primary"":""transmission_repair"",""alternate"":[...",0.77,"[""http://valleytransca.com/""]",None,None,"[""6194474353""]","{""names"":{}}","[{""freeform"":""1158 N 2nd St"",""locality"":""El Ca..."
2,08fbcd0030da5323031bcafa8c2fa0dc,844424934845986,"[{""property"":"""",""dataset"":""meta"",""record_id"":""...","{""primary"":""Mazda Nelspruit""}","{""primary"":""car_dealer"",""alternate"":[""automoti...",0.997899,"[""http://bit.ly/NelspruitMazda""]","[""https://www.facebook.com/113070635372645""]",None,"[""+27137573800""]",...,"[{""dataset"":""Microsoft"",""record_id"":""844424934...","{""primary"":""Mazda Nelspruit""}","{""primary"":""used_car_dealer"",""alternate"":[""aut...",0.77,"[""https://nelspruitmazda.co.za/""]",None,None,"[""0137573800""]","{""names"":{}}","[{""freeform"":""49 Emnotweni Avenue"",""locality"":..."
3,1407374885933937,844424932146366,"[{""dataset"":""Microsoft"",""record_id"":""140737488...","{""primary"":""Red Wing - Roswell, GA""}","{""primary"":""shoe_store"",""alternate"":[""online_s...",0.770000,"[""https://stores.redwingshoes.com/roswell-ga""]",None,None,"[""7705584400""]",...,"[{""dataset"":""Microsoft"",""record_id"":""844424932...","{""primary"":""Red Wing""}","{""primary"":""shoe_store"",""alternate"":[""online_s...",0.77,"[""http://stores.redwing.com/roswell-ga""]",None,None,"[""7705584400""]","{""names"":{}}","[{""freeform"":""875 Mansell Rd Ste B1"",""locality..."
4,08f3956260b9e14003feca2bf0764d0c,1407374887472291,"[{""property"":"""",""dataset"":""meta"",""record_id"":""...","{""primary"":""Norauto España""}","{""primary"":""automotive_repair"",""alternate"":[""c...",0.996261,"[""https://centro.norauto.es/alacant/finestrat/...","[""https://www.facebook.com/295967644448975""]",None,"[""+34966803254""]",...,"[{""dataset"":""Microsoft"",""record_id"":""140737488...","{""primary"":""Norauto""}","{""primary"":""brake_service_and_repair"",""alterna...",0.77,"[""http://centro.norauto.es/centre/alicante-fin...",None,None,"[""966803254""]","{""names"":{}}","[{""freeform"":""Calle Rei Jaume I 9"",""locality"":..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,08f1e1251b0a66c003a195f0bf6b4bc9,502a184a011ce8eabd5934fb,"[{""property"":"""",""dataset"":""meta"",""record_id"":""...","{""primary"":""Studio Legale Associato Petracci M...","{""primary"":""lawyer"",""alternate"":[""criminal_def...",0.286713,"[""http://www.petraccimarin.it""]","[""https://www.facebook.com/121776774503515""]",None,"[""+39040660022""]",...,"[{""property"":"""",""dataset"":""FourSquare"",""record...","{""primary"":""Studio Legale Associato Petracci M...","{""primary"":""Business 

In [75]:
# For previewing the data, look at first data point in sample_df
first_data_point = sample_df.iloc[0]

print("Sources value:")
print(first_data_point['names'])
print(type(first_data_point['names']))

print("\nParsed sources:")
sources_data = json.loads(first_data_point['names'])
for i, source in enumerate(sources_data):
    print(f"Source {i+1}: {source}")

Sources value:
{"primary":"Goin' Postal Jacksonville"}
<class 'str'>

Parsed sources:
Source 1: primary


In [76]:
# Check current types
sample_df.map(type)

,id,base_id,sources,names,categories,confidence,websites,socials,emails,phones,...,base_sources,base_names,base_categories,base_confidence,base_websites,base_socials,base_emails,base_phones,base_brand,base_addresses
0,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'NoneType'>,<class 'NoneType'>,<class 'str'>,<class 'str'>,<class 'str'>
1,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'NoneType'>,<class 'NoneType'>,<class 'str'>,<class 'str'>,<class 'str'>
2,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'NoneType'>,<class 'NoneType'>,<class 'str'>,<class 'str'>,<class 'str'>
3,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'NoneType'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'NoneType'>,<class 'NoneType'>,<class 'str'>,<class 'str'>,<class 'str'>
4,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'NoneType'>,<class 'NoneType'>,<class 'str'>,<class 'str'>,<class 'str'>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
1996,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
1997,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'NoneType'>,<class 'str'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
1998,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>


In [77]:
# Assigns data types to attributes
def parse_category(x):
    if isinstance(x, (list, dict)) or pd.isna(x): return x
    try: return ast.literal_eval(x)
    except: return None

for c in ["sources", "names", "addresses","base_names","base_addresses","base_categories", "categories", "websites", "brand"]:
    if c in sample_df: sample_df[c] = sample_df[c].apply(parse_category)

sample_df.map(type)

,id,base_id,sources,names,categories,confidence,websites,socials,emails,phones,...,base_sources,base_names,base_categories,base_confidence,base_websites,base_socials,base_emails,base_phones,base_brand,base_addresses
0,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'str'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'str'>,<class 'NoneType'>,<class 'NoneType'>,<class 'str'>,<class 'str'>,<class 'list'>
1,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'str'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'str'>,<class 'NoneType'>,<class 'NoneType'>,<class 'str'>,<class 'str'>,<class 'list'>
2,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'str'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'str'>,<class 'NoneType'>,<class 'NoneType'>,<class 'str'>,<class 'str'>,<class 'list'>
3,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'NoneType'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'str'>,<class 'NoneType'>,<class 'NoneType'>,<class 'str'>,<class 'str'>,<class 'list'>
4,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'str'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'str'>,<class 'NoneType'>,<class 'NoneType'>,<class 'str'>,<class 'str'>,<class 'list'>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'str'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>
1996,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'str'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>
1997,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'NoneType'>,<class 'str'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>
1998,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'str'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>


In [78]:
def norm_name(s):
    s = re.sub(r"\s+", " ", (s or "").strip().lower())
    s = re.sub(r"\b(llc|inc|corp|co|ltd|the)\b", "", s)
    s = re.sub(r"[^a-z0-9\s&\-']", " ", s)
    return re.sub(r"\s+", " ", s).strip()

def extract_postcode(addr_list):
    if isinstance(addr_list, list) and addr_list:
        pc = str(addr_list[0].get("postcode") or None)
        if pc and pc != 'None':
            # Remove ZIP+4 extension (e.g., "49007-1293" becomes "49007")
            pc = pc.split('-')[0].strip()
        return pc

def extract_city(addr_list):
    if isinstance(addr_list, list) and addr_list:
        city_str = addr_list[0].get("locality") or None
        if city_str:
            return city_str.strip().lower()
        return city_str

def extract_name(name_list):
    if isinstance(name_list, dict) and name_list:
        return name_list.get("primary")

In [79]:
sample_df

,id,base_id,sources,names,categories,confidence,websites,socials,emails,phones,...,base_sources,base_names,base_categories,base_confidence,base_websites,base_socials,base_emails,base_phones,base_brand,base_addresses
0,08f44f055a9a016e0390f050aa3c93c0,1688849865669487,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Goin' Postal Jacksonville'},"{'primary': 'shipping_center', 'alternate': ['...",0.996261,[http://www.goinpostaljacksonville.com/],"[""https://www.facebook.com/463273470392736""]",None,"[""+19049989600""]",...,"[{""dataset"":""Microsoft"",""record_id"":""168884986...",{'primary': 'Goin' Postal Jacksonville'},"{'primary': 'vehicle_shipping', 'alternate': [...",0.77,"[""https://www.goinpostaljacksonville.com/""]",None,None,"[""9049989600""]","{""names"":{}}","[{'freeform': '7643 Gate Pkwy Ste 104', 'local..."
1,08f29a456e42e5830324637954145c50,1125899907111860,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Valley Transmission'},"{'primary': 'automotive_repair', 'alternate': ...",0.996261,[http://valleytransca.com/],"[""https://www.facebook.com/107150256087765""]",None,"[""+16194474353""]",...,"[{""dataset"":""Microsoft"",""record_id"":""112589990...",{'primary': 'Valley Transmission'},"{'primary': 'transmission_repair', 'alternate'...",0.77,"[""http://valleytransca.com/""]",None,None,"[""6194474353""]","{""names"":{}}","[{'freeform': '1158 N 2nd St', 'locality': 'El..."
2,08fbcd0030da5323031bcafa8c2fa0dc,844424934845986,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Mazda Nelspruit'},"{'primary': 'car_dealer', 'alternate': ['autom...",0.997899,[http://bit.ly/NelspruitMazda],"[""https://www.facebook.com/113070635372645""]",None,"[""+27137573800""]",...,"[{""dataset"":""Microsoft"",""record_id"":""844424934...",{'primary': 'Mazda Nelspruit'},"{'primary': 'used_car_dealer', 'alternate': ['...",0.77,"[""https://nelspruitmazda.co.za/""]",None,None,"[""0137573800""]","{""names"":{}}","[{'freeform': '49 Emnotweni Avenue', 'locality..."
3,1407374885933937,844424932146366,"[{'dataset': 'Microsoft', 'record_id': '140737...","{'primary': 'Red Wing - Roswell, GA'}","{'primary': 'shoe_store', 'alternate': ['onlin...",0.770000,[https://stores.redwingshoes.com/roswell-ga],None,None,"[""7705584400""]",...,"[{""dataset"":""Microsoft"",""record_id"":""844424932...",{'primary': 'Red Wing'},"{'primary': 'shoe_store', 'alternate': ['onlin...",0.77,"[""http://stores.redwing.com/roswell-ga""]",None,None,"[""7705584400""]","{""names"":{}}","[{'freeform': '875 Mansell Rd Ste B1', 'locali..."
4,08f3956260b9e14003feca2bf0764d0c,1407374887472291,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Norauto España'},"{'primary': 'automotive_repair', 'alternate': ...",0.996261,[https://centro.norauto.es/alacant/finestrat/c...,"[""https://www.facebook.com/295967644448975""]",None,"[""+34966803254""]",...,"[{""dataset"":""Microsoft"",""record_id"":""140737488...",{'primary': 'Norauto'},"{'primary': 'brake_service_and_repair', 'alter...",0.77,"[""http://centro.norauto.es/centre/alicante-fin...",None,None,"[""966803254""]","{""names"":{}}","[{'freeform': 'Calle Rei Jaume I 9', 'locality..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,08f1e1251b0a66c003a195f0bf6b4bc9,502a184a011ce8eabd5934fb,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Studio Legale Associato Petracci ...,"{'primary': 'lawyer', 'alternate': ['criminal_...",0.286713,[http://www.petraccimarin.it],"[""https://www.facebook.com/121776774503515""]",None,"[""+39040660022""]",...,"[{""property"":"""",""dataset"":""FourSquare"",""record...",{'primary': 'Studio Legale Associato Petracci ...,{'primary': 'Business and Professional Service...,1.00,"[""http://www.petraccimarin.it""]","[null,null,null]",[null],"[""040 660022""]","{""wikidata"":"""",""names"":{""primary"":""Studio Lega...","[{'freeform': 'Via beccaria 3', 'locality': 'T..."
1996,08fa80b035674b6a03379ca9c1e

# Attribute Conflation Labeling Strategy

The `sample_df` dataset contains pairs of places where each row represents the same physical place from two different data sources. The "base_" prefixed columns represent Place A (source A) and the non-prefixed columns represent Place B (source B). 

Goal: Create a labeling system to determine which attributes are more reliable for building a universal places database.

## Strategy Overview

Two labeling approaches:

1. **Per-Attribute Binary Labels**: For each attribute type, determine if Place A or Place B has the better/more reliable value
2. **Overall Quality Score**: An aggregate score indicating which place (A or B) has better overall data quality

## Key Attributes to Label:
- Names
- Addresses 
- Categories
- Websites
- Socials
- Emails
- Phones
- Brand

In [80]:
# Labeling Strategy Implementation

class AttributeQualityEvaluator:
    """
    Evaluates quality of attributes for place conflation labeling
    """
    
    def __init__(self):
        self.attribute_weights = {
            'names': 0.25,
            'addresses': 0.20, 
            'categories': 0.15,
            'websites': 0.10,
            'phones': 0.10,
            'emails': 0.10,
            'socials': 0.05,
            'brand': 0.05
        }
    
    def evaluate_name_quality(self, name_data):
        """Evaluate name quality based on completeness and structure"""
        if not name_data or pd.isna(name_data):
            return 0.0
            
        if isinstance(name_data, dict):
            score = 0.0
            # Primary name exists
            if name_data.get('primary'):
                score += 0.6
                # Bonus for reasonable length
                if 5 <= len(name_data['primary']) <= 100:
                    score += 0.2
            # Alternative names exist
            if name_data.get('alternatives') and len(name_data['alternatives']) > 0:
                score += 0.2
            return min(score, 1.0)
        
        # If it's a string, basic scoring
        if isinstance(name_data, str):
            if 5 <= len(name_data) <= 100:
                return 0.8
            return 0.4
        
        return 0.0
    
    def evaluate_address_quality(self, address_data):
        """Evaluate address quality based on completeness and structure"""
        if not address_data or pd.isna(address_data):
            return 0.0
            
        if isinstance(address_data, list) and len(address_data) > 0:
            addr = address_data[0]
            score = 0.0
            
            # Street address component
            if addr.get('street'):
                score += 0.3
            
            # City component
            if addr.get('locality') or addr.get('city'):
                score += 0.25
            
            # Postal code component
            if addr.get('postcode'):
                pc = str(addr.get('postcode'))
                # Bonus for valid format
                if re.match(r'^\d{5}(-\d{4})?$', pc):  # US ZIP
                    score += 0.25
                else:
                    score += 0.15
            
            # Country component
            if addr.get('country'):
                score += 0.1
            
            # State/region component
            if addr.get('region') or addr.get('state'):
                score += 0.1
            
            return min(score, 1.0)
        
        return 0.0
    
    def evaluate_categories_quality(self, categories_data):
        """Evaluate categories quality based on number and specificity"""
        if not categories_data or pd.isna(categories_data):
            return 0.0
        
        if isinstance(categories_data, list):
            if len(categories_data) == 0:
                return 0.0
            
            score = 0.0
            # Base score for having categories
            score += 0.4
            
            # Bonus for multiple categories (up to a point)
            num_cats = min(len(categories_data), 5)
            score += (num_cats - 1) * 0.1
            
            # Bonus for specific categories (longer names often more specific)
            avg_length = np.mean([len(str(cat)) for cat in categories_data])
            if avg_length > 10:
                score += 0.2
            
            return min(score, 1.0)
        
        return 0.0
    
    def evaluate_website_quality(self, website_data):
        """Evaluate website quality based on validity and completeness"""
        if not website_data or pd.isna(website_data):
            return 0.0
        
        if isinstance(website_data, list):
            valid_urls = 0
            for url in website_data:
                if self.is_valid_url(url):
                    valid_urls += 1
            
            if valid_urls > 0:
                return min(0.5 + (valid_urls - 1) * 0.2, 1.0)
        
        return 0.0
    
    def evaluate_contact_quality(self, contact_data, contact_type='phone'):
        """Evaluate phone/email quality"""
        if not contact_data or pd.isna(contact_data):
            return 0.0
        
        if isinstance(contact_data, list):
            valid_contacts = 0
            for contact in contact_data:
                if contact_type == 'phone' and self.is_valid_phone(contact):
                    valid_contacts += 1
                elif contact_type == 'email' and self.is_valid_email(contact):
                    valid_contacts += 1
            
            if valid_contacts > 0:
                return min(0.6 + (valid_contacts - 1) * 0.2, 1.0)
        
        return 0.0
    
    def evaluate_social_quality(self, social_data):
        """Evaluate social media quality"""
        if not social_data or pd.isna(social_data):
            return 0.0
        
        if isinstance(social_data, list) and len(social_data) > 0:
            return min(0.3 + len(social_data) * 0.2, 1.0)
        
        return 0.0
    
    def evaluate_brand_quality(self, brand_data):
        """Evaluate brand quality"""
        if not brand_data or pd.isna(brand_data):
            return 0.0
        
        if isinstance(brand_data, str) and len(brand_data) > 2:
            return 0.8
        
        return 0.0
    
    def is_valid_url(self, url):
        """Check if URL is valid"""
        try:
            result = urlparse(str(url))
            return all([result.scheme, result.netloc])
        except:
            return False
    
    def is_valid_phone(self, phone):
        """Check if phone number looks valid"""
        if not phone:
            return False
        
        # Remove common formatting
        clean_phone = re.sub(r'[^\d+]', '', str(phone))
        # Basic validation - reasonable length
        return 7 <= len(clean_phone) <= 15
    
    def is_valid_email(self, email):
        """Check if email looks valid"""
        if not email:
            return False
        
        email_pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
        return bool(re.match(email_pattern, str(email)))

# Initialize evaluator
evaluator = AttributeQualityEvaluator()

In [81]:
def generate_attribute_labels(row):
    """
    Generate per-attribute and overall quality labels for a place pair
    
    Returns:
    - Per-attribute labels: 1 if Place A (base_) is better, 0 if Place B is better
    - Overall label: 1 if Place A is better overall, 0 if Place B is better overall
    - Quality scores for transparency
    """
    
    labels = {}
    place_a_scores = {}
    place_b_scores = {}
    
    # Evaluate each attribute pair
    attribute_pairs = [
        ('names', 'base_names'),
        ('addresses', 'base_addresses'), 
        ('categories', 'base_categories'),
        ('websites', 'base_websites'),
        ('phones', 'base_phones'),
        ('emails', 'base_emails'),
        ('socials', 'base_socials'),
        ('brand', 'base_brand')
    ]
    
    for attr_b, attr_a in attribute_pairs:
        if attr_a in row.index and attr_b in row.index:
            # Get quality scores
            if attr_b == 'names':
                score_a = evaluator.evaluate_name_quality(row[attr_a])
                score_b = evaluator.evaluate_name_quality(row[attr_b])
            elif attr_b == 'addresses':
                score_a = evaluator.evaluate_address_quality(row[attr_a])
                score_b = evaluator.evaluate_address_quality(row[attr_b])
            elif attr_b == 'categories':
                score_a = evaluator.evaluate_categories_quality(row[attr_a])
                score_b = evaluator.evaluate_categories_quality(row[attr_b])
            elif attr_b == 'websites':
                score_a = evaluator.evaluate_website_quality(row[attr_a])
                score_b = evaluator.evaluate_website_quality(row[attr_b])
            elif attr_b in ['phones', 'emails']:
                contact_type = 'phone' if attr_b == 'phones' else 'email'
                score_a = evaluator.evaluate_contact_quality(row[attr_a], contact_type)
                score_b = evaluator.evaluate_contact_quality(row[attr_b], contact_type)
            elif attr_b == 'socials':
                score_a = evaluator.evaluate_social_quality(row[attr_a])
                score_b = evaluator.evaluate_social_quality(row[attr_b])
            elif attr_b == 'brand':
                score_a = evaluator.evaluate_brand_quality(row[attr_a])
                score_b = evaluator.evaluate_brand_quality(row[attr_b])
            else:
                score_a = score_b = 0.0
            
            # Store scores
            place_a_scores[attr_b] = score_a
            place_b_scores[attr_b] = score_b
            
            # Create binary label (1 if A is better, 0 if B is better)
            # If scores are very close (within 0.1), prefer the one with actual data
            if abs(score_a - score_b) < 0.1:
                # Tie-breaking: prefer non-null data
                has_data_a = row[attr_a] is not None and not pd.isna(row[attr_a])
                has_data_b = row[attr_b] is not None and not pd.isna(row[attr_b])
                
                if has_data_a and not has_data_b:
                    labels[f'label_{attr_b}'] = 1
                elif has_data_b and not has_data_a:
                    labels[f'label_{attr_b}'] = 0
                else:
                    # True tie - slight preference for A (base)
                    labels[f'label_{attr_b}'] = 1
            else:
                labels[f'label_{attr_b}'] = 1 if score_a > score_b else 0
    
    # Calculate overall weighted score
    total_weight_a = 0
    total_weight_b = 0
    
    for attr_b in place_a_scores.keys():
        if attr_b in evaluator.attribute_weights:
            weight = evaluator.attribute_weights[attr_b]
            total_weight_a += weight * place_a_scores[attr_b]
            total_weight_b += weight * place_b_scores[attr_b]
    
    # Overall label
    labels['label_overall'] = 1 if total_weight_a > total_weight_b else 0
    
    # Add quality scores for transparency
    labels['quality_score_a'] = total_weight_a
    labels['quality_score_b'] = total_weight_b
    labels['quality_difference'] = total_weight_a - total_weight_b
    
    return pd.Series(labels)

# Test the labeling function on first few rows
print("Testing labeling function on first 3 rows:")
test_labels = sample_df.head(3).apply(generate_attribute_labels, axis=1)
print(test_labels)

Testing labeling function on first 3 rows:
   label_names  label_addresses  label_categories  label_websites  \
0          1.0              1.0               1.0             0.0   
1          1.0              1.0               1.0             0.0   
2          1.0              1.0               1.0             0.0   

   label_phones  label_emails  label_socials  label_brand  label_overall  \
0           1.0           1.0            0.0          1.0            0.0   
1           1.0           1.0            0.0          1.0            0.0   
2           1.0           1.0            0.0          1.0            0.0   

   quality_score_a  quality_score_b  quality_difference  
0             0.38             0.39               -0.01  
1             0.38             0.39               -0.01  
2             0.34             0.37               -0.03  


In [82]:
# Apply labeling to entire dataset
print("Generating labels for all 2000 place pairs...")
start_time = time.time()

# Generate labels for all rows
labels_df = sample_df.apply(generate_attribute_labels, axis=1)

# Combine with original data
labeled_sample_df = pd.concat([sample_df, labels_df], axis=1)

end_time = time.time()
print(f"Labeling completed in {end_time - start_time:.2f} seconds")

# Display labeling summary
print("\n=== LABELING SUMMARY ===")
print(f"Total place pairs labeled: {len(labeled_sample_df)}")

print("\nPer-attribute label distribution:")
attribute_labels = [col for col in labels_df.columns if col.startswith('label_') and col != 'label_overall']
for label_col in attribute_labels:
    attr_name = label_col.replace('label_', '')
    count_a = (labels_df[label_col] == 1).sum()
    count_b = (labels_df[label_col] == 0).sum()
    print(f"  {attr_name:12s}: Place A (base_) better: {count_a:4d} | Place B better: {count_b:4d}")

print(f"\nOverall quality distribution:")
overall_a = (labels_df['label_overall'] == 1).sum()
overall_b = (labels_df['label_overall'] == 0).sum()
print(f"  Place A (base_) better overall: {overall_a}")
print(f"  Place B better overall: {overall_b}")

print(f"\nQuality score statistics:")
print(f"  Place A mean quality: {labels_df['quality_score_a'].mean():.3f}")
print(f"  Place B mean quality: {labels_df['quality_score_b'].mean():.3f}")
print(f"  Mean quality difference (A-B): {labels_df['quality_difference'].mean():.3f}")
print(f"  Quality difference std: {labels_df['quality_difference'].std():.3f}")

Generating labels for all 2000 place pairs...
Labeling completed in 0.22 seconds

=== LABELING SUMMARY ===
Total place pairs labeled: 2000

Per-attribute label distribution:
  names       : Place A (base_) better: 1970 | Place B better:   30
  addresses   : Place A (base_) better: 1865 | Place B better:  135
  categories  : Place A (base_) better: 2000 | Place B better:    0
  websites    : Place A (base_) better:  339 | Place B better: 1661
  phones      : Place A (base_) better: 1999 | Place B better:    1
  emails      : Place A (base_) better: 2000 | Place B better:    0
  socials     : Place A (base_) better: 1466 | Place B better:  534
  brand       : Place A (base_) better: 1995 | Place B better:    5

Overall quality distribution:
  Place A (base_) better overall: 598
  Place B better overall: 1402

Quality score statistics:
  Place A mean quality: 0.364
  Place B mean quality: 0.363
  Mean quality difference (A-B): 0.001
  Quality difference std: 0.030


In [83]:
# Create detailed analysis and examples

# 1. Show examples of clear winners
print("=== EXAMPLES OF CLEAR QUALITY DIFFERENCES ===")

# Find rows with largest quality differences
top_a_indices = labels_df.nlargest(3, 'quality_difference').index
top_b_indices = labels_df.nsmallest(3, 'quality_difference').index

print("\nTop 3 cases where Place A (base_) is clearly better:")
for i, idx in enumerate(top_a_indices):
    row = labeled_sample_df.iloc[idx]
    print(f"\nExample {i+1} (Row {idx}):")
    print(f"  Quality difference: {row['quality_difference']:.3f}")
    print(f"  Place A name: {row['base_names']}")
    print(f"  Place B name: {row['names']}")
    print(f"  Place A address: {row['base_addresses']}")
    print(f"  Place B address: {row['addresses']}")

print("\n\nTop 3 cases where Place B is clearly better:")
for i, idx in enumerate(top_b_indices):
    row = labeled_sample_df.iloc[idx]
    print(f"\nExample {i+1} (Row {idx}):")
    print(f"  Quality difference: {row['quality_difference']:.3f}")
    print(f"  Place A name: {row['base_names']}")
    print(f"  Place B name: {row['names']}")
    print(f"  Place A websites: {row['base_websites']}")
    print(f"  Place B websites: {row['websites']}")

# 2. Analyze attribute correlation
print("\n\n=== ATTRIBUTE CORRELATION ANALYSIS ===")

# Create correlation matrix for binary labels
label_cols = [col for col in labels_df.columns if col.startswith('label_') and col != 'label_overall']
correlation_matrix = labels_df[label_cols].corr()

print("Correlation between attribute labels:")
print(correlation_matrix.round(3))

# 3. Quality score distribution
print(f"\n=== QUALITY SCORE DISTRIBUTION ===")
print(f"Quality difference distribution:")
print(f"  Min: {labels_df['quality_difference'].min():.3f}")
print(f"  25th percentile: {labels_df['quality_difference'].quantile(0.25):.3f}")
print(f"  Median: {labels_df['quality_difference'].median():.3f}")
print(f"  75th percentile: {labels_df['quality_difference'].quantile(0.75):.3f}")
print(f"  Max: {labels_df['quality_difference'].max():.3f}")

# 4. Identify challenging cases (very close quality scores)
close_cases = labeled_sample_df[abs(labeled_sample_df['quality_difference']) < 0.01]
print(f"\nChallenging cases (quality difference < 0.01): {len(close_cases)} out of {len(labeled_sample_df)}")

=== EXAMPLES OF CLEAR QUALITY DIFFERENCES ===

Top 3 cases where Place A (base_) is clearly better:

Example 1 (Row 1243):
  Quality difference: 0.160
  Place A name: {'primary': '粟生団地バス停', 'common': {}, 'rules': []}
  Place B name: {'primary': '粟生団地'}
  Place A address: [{'freeform': '粟生間谷西2丁目9', 'locality': '箕面市', 'region': '大阪府', 'country': 'JP', 'postcode': '562-0023'}]
  Place B address: [{'freeform': '粟生間谷西2丁目9付近', 'postcode': '562-0023', 'country': 'JP'}]

Example 2 (Row 1693):
  Quality difference: 0.160
  Place A name: {'primary': 'จิ๋ม&แดง (Jim&Dang Seafood)', 'common': {}, 'rules': []}
  Place B name: {'primary': 'จิ๋ม&แดง'}
  Place A address: [{'freeform': '27 หมู่ที่ 2', 'locality': 'Sam Roi Yot', 'region': 'Prachuap Khiri Khan', 'country': 'TH', 'postcode': '77120'}]
  Place B address: [{'freeform': '27 หมู่ที่ 2', 'country': 'TH'}]

Example 3 (Row 1425):
  Quality difference: 0.120
  Place A name: {'primary': '東名吉田バス停', 'common': {}, 'rules': []}
  Place B name: {'primar

In [84]:
# Export labeled dataset and create usage guidelines

def export_labeled_dataset(df, filename='labeled_places_pairs.csv'):
    """Export the labeled dataset with comprehensive metadata"""
    
    # Create metadata dictionary
    metadata = {
        'creation_date': pd.Timestamp.now().isoformat(),
        'total_pairs': len(df),
        'labeling_strategy': 'quality_based_binary_labels',
        'label_types': {
            'per_attribute_labels': 'Binary (1=Place A better, 0=Place B better)',
            'overall_label': 'Binary (1=Place A better overall, 0=Place B better overall)',
            'quality_scores': 'Continuous (0-1 scale)'
        },
        'attributes_evaluated': [
            'names', 'addresses', 'categories', 'websites', 
            'phones', 'emails', 'socials', 'brand'
        ],
        'weighting_scheme': {
            'names': 0.25,
            'addresses': 0.20,
            'categories': 0.15,
            'websites': 0.10,
            'phones': 0.10,
            'emails': 0.10,
            'socials': 0.05,
            'brand': 0.05
        }
    }
    
    # Select columns to export (avoid complex nested structures)
    export_columns = ['id', 'base_id'] + [col for col in df.columns if col.startswith('label_') or col.startswith('quality_')]
    simplified_df = df[export_columns].copy()
    
    # Save dataset
    simplified_df.to_csv(filename, index=False)
    
    # Save metadata
    metadata_filename = filename.replace('.csv', '_metadata.json')
    with open(metadata_filename, 'w') as f:
        json.dump(metadata, f, indent=2)
    
    print(f"Labeled dataset exported to: {filename}")
    print(f"Metadata exported to: {metadata_filename}")
    print(f"Exported {len(simplified_df)} rows with {len(export_columns)} columns")
    
    return metadata

# Export the labeled dataset
metadata = export_labeled_dataset(labeled_sample_df)

# Show what was exported
print(f"\nExported columns:")
export_columns = ['id', 'base_id'] + [col for col in labeled_sample_df.columns if col.startswith('label_') or col.startswith('quality_')]
for col in export_columns:
    print(f"  {col}")

Labeled dataset exported to: labeled_places_pairs.csv
Metadata exported to: labeled_places_pairs_metadata.json
Exported 2000 rows with 14 columns

Exported columns:
  id
  base_id
  label_names
  label_addresses
  label_categories
  label_websites
  label_phones
  label_emails
  label_socials
  label_brand
  label_overall
  quality_score_a
  quality_score_b
  quality_difference
